In [5]:
# libraries

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

import pandas as pd
import re
from deep_translator import GoogleTranslator
import tqdm
import os

In [6]:
# helper functions:
def myTranslate(t):
    # Translate to English:
    text = GoogleTranslator(source='auto', target='en').translate(t)
    return text
    
def extractNumber(t):
    phone = re.findall(r'[[0-9][0-9 .\-\(\)]{6,}[0-9]', t)
    return phone

def getLabelData(elem):
    ts = elem.text.split("\n")
    name = ts[0]
    
    # TODO 
#     name = myTranslate(name)
    
    reviews = ts[1]
    try:
        address = ts[2]
    except:
        address = ""
        
#     address = myTranslate(address)

#     temp = name + " || " + address
#     trans = myTranslate(temp)
#     try:
#         name,address = trans.split("||")
#     except:
#         print("Issue with splitting ::",trans)
#         name = myTranslate(name)
#         address = myTranslate(address)
    
    
    # try replace
    address = address.replace("Pharmaceutical company ·","")
    
    try:
        timings_phone = ts[3]
    except:
        timings_phone = ""
        
    temp = extractNumber(timings_phone)
    if len(temp)>0:
        phonenumber = temp[0]
    else:
        phonenumber = ""
    # adding website
    
    try:
        a = elem.find_element(By.TAG_NAME,"a")
        website = a.get_attribute('href')
    except:
        website = ""
        
    obj = {
        "name":name.strip(),
        "phone":phonenumber,
        "website":website,
        "address":address.strip()
        
    }
    return obj


In [7]:
# Initiating the browser
options = webdriver.ChromeOptions()
path = "../chrome/chromedriver.exe"
service = Service(executable_path=path)
browser = webdriver.Chrome(service=service,options=options)
browser.maximize_window()

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\yeshu\AppData\Local\Temp\ipykernel_7580\3017798992.py:3: SyntaxWarning: invalid escape sequence '\c'
  path = "../chrome\chromedriver.exe"


### Step 1: Industry Selection

In [8]:
industry = "Food"

### 1. High level search:

In [9]:
country = "India" # Please change accordingly

In [11]:
company_category = f"{industry} Companies"
url = "https://www.google.com/maps/search/{} in {}".format(company_category, country)
company_category, url

('Food Companies',
 'https://www.google.com/maps/search/Food Companies in India')

In [12]:
data = []

In [13]:
# started extracting
print("Currently extracting from :: ",url)

# Open the Google Map URL
browser.get(url)
# scrolling : 
actionChains = ActionChains(browser)

# scrolling upto 20 times::
end = False
repeat = 0
limit = 28
    
while not end and repeat < limit:

    # check end
    end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
    if len(end_identify)>0:
        end = True
        print("Reached end !!")

    repeat += 1
    if repeat == limit:
        print("28 times reached !!")
        end = True
    
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    if len(elems)==0:
        print("Remember to add ")
        continue
    elem = elems[-1]
    
    xoffset = 0
    yoffset = 80 # If you get "target move out of boundaries" => change this value
    
    actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
    browser.execute_script("arguments[0].scrollIntoView();", elem)




Currently extracting from ::  https://www.google.com/maps/search/Food Companies in India
28 times reached !!


In [14]:
# extracting the data
elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
for elem in elems:
    d = getLabelData(elem)
    data.append(d)


C:\Users\yeshu\AppData\Local\Temp\ipykernel_7580\595260018.py:8: FutureWarning: Possible nested set at position 1
  phone = re.findall(r'[[0-9][0-9 .\-\(\)]{6,}[0-9]', t)


In [15]:
## saving results:
df = pd.DataFrame(data)
df.head()

,name,phone,website,address
0,FNF Industries Limited,040 6668 8945,,"Food Processing Company · # 10-1-120, 2nd Floo..."
1,The Superfood Company,099009 96631,,"Food manufacturer · RAINBOW VISTAS PHASE-1, Gr..."
2,Ambika Food Industries Pvt. Ltd.,040 6668 8890,http://www.ambikafood.in/,"Food products supplier · 15-9-583/584, Mehboob..."
3,Deccan Food Products,094920 09807,,"Food Processing Company · Daulat Market, 15-7-..."
4,Cyberabad Foods PVT LTD,098854 04440,,Food manufacturer · G6PX+5Q9


In [17]:
# creating a folder
os.mkdir("../results/industries/{}".format(industry))

In [18]:
os.mkdir("../results/industries/{}/{}".format(industry,country))

In [19]:
df.to_csv("../results/industries/{}/{}/names/companies_highlevel_1.csv".format(industry,country),index=None)

#### Cites wise

In [20]:
# loading the list of cities
cities_df = pd.read_csv("../data/worldcities.csv")
cities = list(cities_df[cities_df['country'] == country]['city_ascii'].values)

In [21]:
len(cities)

5933

In [22]:
cities

['Delhi',
 'Mumbai',
 'Kolkata',
 'Bangalore',
 'Chennai',
 'Hyderabad',
 'Pune',
 'Ahmedabad',
 'Surat',
 'Prayagraj',
 'Lucknow',
 'Jaipur',
 'Cawnpore',
 'Mirzapur',
 'Nagpur',
 'Ghaziabad',
 'Vadodara',
 'Vishakhapatnam',
 'Indore',
 'Thane',
 'Bhopal',
 'Chinchvad',
 'Patna',
 'Bilaspur',
 'Ludhiana',
 'Agra',
 'Agwar',
 'Madurai',
 'Jamshedpur',
 'Nasik',
 'Faridabad',
 'Aurangabad',
 'Rajkot',
 'Meerut',
 'Jabalpur',
 'Kalamboli',
 'Vasai',
 'Najafgarh',
 'Varanasi',
 'Srinagar',
 'Dhanbad',
 'Amritsar',
 'Aligarh',
 'Guwahati',
 'Haora',
 'Ranchi',
 'Gwalior',
 'Vijayawada',
 'Chandigarh',
 'Jodhpur',
 'Raipur',
 'Kota',
 'New Delhi',
 'Bareilly',
 'Coimbatore',
 'Solapur',
 'Trichinopoly',
 'Moradabad',
 'Tiruppur',
 'Gurgaon',
 'Jalandhar',
 'Kalkaji Devi',
 'Bhubaneshwar',
 'Bhayandar',
 'Mandvi',
 'Patiala',
 'Dhulia',
 'Thiruvananthapuram',
 'Bhiwandi',
 'Saharanpur',
 'Warangal',
 'Shiliguri',
 'Salem',
 'Vellore',
 'Kochi',
 'Gorakhpur',
 'Narela',
 'Guntur',
 'Rasulpur'

In [23]:
## Remember to drag , zoom in and then extract : repeat
## Initiating the browser
options = webdriver.ChromeOptions()
path = "../chrome/chromedriver.exe"
service = Service(executable_path=path)
browser = webdriver.Chrome(service=service,options=options)
browser.maximize_window()

# browser.set_page_load_timeout(300) # only for 5 minutes

In [24]:
for city in tqdm.tqdm(cities[:7]):
    data = []
    url = "https://www.google.com/maps/search/{} companies in {} {}".format(industry, city, country)
#     print("Currently extracting from :: ",url)
    browser.get(url)
    
    # scrolling : 
    actionChains = ActionChains(browser)

    # scrolling upto 20 times::
    end = False
    repeat = 0
    limit = 28

    while not end and repeat < limit:

        # check end
        end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
        if len(end_identify)>0:
            end = True
#             print("Reached end !!")

        repeat += 1
        if repeat == limit:
#             print("28 times reached !!")
            end = True

        elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
        if len(elems)==0:
#             print("Remember to add ")
            continue
        elem = elems[-1]

        xoffset = 0
        yoffset = 80

        actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
        browser.execute_script("arguments[0].scrollIntoView();", elem)
        time.sleep(0.2)


    # extracting the data
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    for elem in elems:
        d = getLabelData(elem)
        data.append(d)

#     print("Done")
#     print("Extracted :: ",len(elems))
    
#     print("Saving the file:")
    
    df = pd.DataFrame(data)
    df.to_csv("../results/industries/{}/{}/names/{}.csv".format(industry,country,city),index=None)
    

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [02:23<00:00, 20.49s/it]


In [ ]:
# Note: There are times when you get Aw, Swap error from google maps: then do below
# Check the number of iteration done till now 
# continue iterating cities from that city
# Run again

#### Selected City - Do Zoom in | Drag Drop | Extract || Repeat

In [23]:
data = []

In [24]:
city = "Riyadh" # Mostly do it for capital city
url = "https://www.google.com/maps/search/Pharmaceutical companies in {} {}".format(city,country)
#     print("Currently extracting from :: ",url)
browser.get(url)

In [25]:
## zoom in 
zoom_button = browser.find_element("xpath",'.//button[@id="widget-zoom-in"]')
action = ActionChains(browser)
action.click(on_element = zoom_button)
action.perform() 

In [26]:
## search this area
search_this_area = browser.find_element("xpath",'.//span[@class="Kw7Rxf"]')
action.click(on_element = search_this_area)
action.perform() 

In [28]:
## Extracting again
# scrolling : 
actionChains = ActionChains(browser)

# scrolling upto 20 times::
end = False
repeat = 0
limit = 28
    
while not end and repeat < limit:

    # check end
    end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
    if len(end_identify)>0:
        end = True
        print("Reached end !!")

    repeat += 1
    if repeat == limit:
        print("28 times reached !!")
        end = True
    
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    if len(elems)==0:
        print("Remember to add ")
        continue
    elem = elems[-1]
    
    xoffset = 0
    yoffset = 80
    
    actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
    browser.execute_script("arguments[0].scrollIntoView();", elem)
    time.sleep(0.2)


# extracting the data
elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
for elem in elems:
    d = getLabelData(elem)
    data.append(d)
    
print("Done")
print("Extracted :: ",len(elems))

Reached end !!
Done
Extracted ::  5


In [29]:
# filtering duplicates:
## removing duplicates
final_data = []
for d in data:
    if d not in final_data:
        final_data.append(d)

In [30]:
len(final_data)

21

In [31]:
df = pd.DataFrame(final_data)

In [32]:
df.to_csv("../results/names/{}/Companies_Rest_All.csv".format(country,city),index=None)